In [7]:
class QueueUsingList:
    def __init__(self):
        self.queue = []
    
    def enqueue(self, item):
        """Add item to rear - O(1) amortized"""
        self.queue.append(item)
    
    def dequeue(self):
        """Remove and return front item - O(n)"""
        if self.is_empty():
            raise IndexError("Dequeue from empty queue")
        return self.queue.pop(0)
    
    def front(self):
        """Return front item without removing - O(1)"""
        if self.is_empty():
            return None
        return self.queue[0]
    
    def is_empty(self):
        """Check if queue is empty - O(1)"""
        return len(self.queue) == 0
    
    def size(self):
        """Return number of items - O(1)"""
        return len(self.queue)
    
    def display(self):
        """Display queue from front to rear"""
        print("Front ->", end=" ")
        for item in self.queue:
            print(item, end=" -> ")
        print("Rear")

In [8]:
from collections import deque

class QueueUsingDeque:
    def __init__(self):
        self.queue = deque()
    
    def enqueue(self, item):
        """Add item to rear - O(1)"""
        self.queue.append(item)
    
    def dequeue(self):
        """Remove and return front item - O(1)"""
        if self.is_empty():
            raise IndexError("Dequeue from empty queue")
        return self.queue.popleft()
    
    def front(self):
        """Return front item without removing - O(1)"""
        if self.is_empty():
            return None
        return self.queue[0]
    
    def is_empty(self):
        """Check if queue is empty - O(1)"""
        return len(self.queue) == 0
    
    def size(self):
        """Return number of items - O(1)"""
        return len(self.queue)
    
    def display(self):
        """Display queue from front to rear"""
        print("Front ->", end=" ")
        for item in self.queue:
            print(item, end=" -> ")
        print("Rear")

# Project 1: Printer Job Management System

In [9]:
import time
import threading
from datetime import datetime

class PrintJob:
    def __init__(self, job_id, document_name, pages, priority=1):
        self.job_id = job_id
        self.document_name = document_name
        self.pages = pagesS
        self.priority = priority  # 1 (highest) to 5 (lowest)
        self.submitted_at = datetime.now()
        self.started_at = None
        self.completed_at = None
        self.status = "Queued"  # Queued, Printing, Completed, Failed
    
    def __str__(self):
        return f"Job {self.job_id}: {self.document_name} ({self.pages} pages, Priority: {self.priority})"
    
    def get_wait_time(self):
        if self.started_at:
            return (self.started_at - self.submitted_at).total_seconds()
        return (datetime.now() - self.submitted_at).total_seconds()
    
    def get_print_time(self):
        if self.completed_at and self.started_at:
            return (self.completed_at - self.started_at).total_seconds()
        return None


class Printer:
    def __init__(self, name, pages_per_minute=10):
        self.name = name
        self.pages_per_minute = pages_per_minute
        self.current_job = None
        self.is_online = True
        self.total_jobs_printed = 0
        self.total_pages_printed = 0
    
    def calculate_print_time(self, pages):
        """Calculate print time in seconds for given pages"""
        return (pages / self.pages_per_minute) * 60
    
    def print_job(self, job):
        """Simulate printing a job"""
        self.current_job = job
        job.status = "Printing"
        job.started_at = datetime.now()
        
        print_time = self.calculate_print_time(job.pages)
        print(f"Printer '{self.name}': Printing {job.document_name} "
              f"({job.pages} pages, est. {print_time:.1f}s)")
        
        # Simulate printing time
        time.sleep(print_time)
        
        job.completed_at = datetime.now()
        job.status = "Completed"
        self.total_jobs_printed += 1
        self.total_pages_printed += job.pages
        
        print(f"Printer '{self.name}': Completed {job.document_name}")
        self.current_job = None
    
    def get_status(self):
        status = f"Printer '{self.name}': "
        if self.current_job:
            status += f"Printing {self.current_job.document_name}"
        else:
            status += "Idle"
        status += f" | Total jobs: {self.total_jobs_printed}, Total pages: {self.total_pages_printed}"
        return status


class PrintQueueManager:
    def __init__(self, num_printers=2):
        self.job_queue = QueueUsingDeque()  # Using deque for efficiency
        self.priority_queue = QueueUsingDeque()  # For high priority jobs
        self.completed_jobs = []
        self.printers = [Printer(f"Printer-{i+1}") for i in range(num_printers)]
        self.job_counter = 1
        self.running = False
        self.print_threads = []
    
    def submit_job(self, document_name, pages, priority=3):
        """Submit a new print job"""
        job = PrintJob(self.job_counter, document_name, pages, priority)
        self.job_counter += 1
        
        if priority <= 2:  # High priority
            self.priority_queue.enqueue(job)
            print(f"Submitted HIGH PRIORITY job: {job}")
        else:
            self.job_queue.enqueue(job)
            print(f"Submitted job: {job}")
        
        self._print_queue_status()
        return job.job_id
    
    def _print_queue_status(self):
        """Display current queue status"""
        total_waiting = self.priority_queue.size() + self.job_queue.size()
        print(f"Queue status: {total_waiting} jobs waiting "
              f"({self.priority_queue.size()} high priority, "
              f"{self.job_queue.size()} normal)")
    
    def get_next_job(self):
        """Get next job to print (priority first)"""
        if not self.priority_queue.is_empty():
            return self.priority_queue.dequeue()
        elif not self.job_queue.is_empty():
            return self.job_queue.dequeue()
        return None
    
    def printer_worker(self, printer):
        """Worker function for each printer"""
        while self.running:
            job = self.get_next_job()
            if job:
                try:
                    printer.print_job(job)
                    self.completed_jobs.append(job)
                except Exception as e:
                    print(f"Printer '{printer.name}' error: {e}")
                    job.status = "Failed"
            else:
                # No jobs, wait a bit
                time.sleep(1)
    
    def start_printing(self):
        """Start the printing system with multiple printers"""
        self.running = True
        self.print_threads = []
        
        for printer in self.printers:
            thread = threading.Thread(target=self.printer_worker, args=(printer,))
            thread.daemon = True
            thread.start()
            self.print_threads.append(thread)
        
        print(f"\nStarted {len(self.printers)} printer(s)")
    
    def stop_printing(self):
        """Stop the printing system"""
        self.running = False
        print("\nStopping printing system...")
    
    def cancel_job(self, job_id):
        """Cancel a queued job by ID"""
        # Check priority queue
        temp_queue = QueueUsingDeque()
        found = False
        
        while not self.priority_queue.is_empty():
            job = self.priority_queue.dequeue()
            if job.job_id == job_id:
                print(f"Cancelled high priority job: {job}")
                found = True
                break
            temp_queue.enqueue(job)
        
        # Restore priority queue
        while not temp_queue.is_empty():
            self.priority_queue.enqueue(temp_queue.dequeue())
        
        if found:
            return True
        
        # Check normal queue
        while not self.job_queue.is_empty():
            job = self.job_queue.dequeue()
            if job.job_id == job_id:
                print(f"Cancelled job: {job}")
                return True
            temp_queue.enqueue(job)
        
        # Restore normal queue
        while not temp_queue.is_empty():
            self.job_queue.enqueue(temp_queue.dequeue())
        
        print(f"Job ID {job_id} not found in queue")
        return False
    
    def get_queue_stats(self):
        """Get statistics about the queue"""
        total_wait_time = 0
        total_print_time = 0
        
        for job in self.completed_jobs:
            total_wait_time += job.get_wait_time()
            print_time = job.get_print_time()
            if print_time:
                total_print_time += print_time
        
        stats = {
            'total_jobs_submitted': self.job_counter - 1,
            'total_jobs_completed': len(self.completed_jobs),
            'total_jobs_waiting': self.priority_queue.size() + self.job_queue.size(),
            'avg_wait_time': total_wait_time / len(self.completed_jobs) if self.completed_jobs else 0,
            'avg_print_time': total_print_time / len(self.completed_jobs) if self.completed_jobs else 0,
            'total_pages_printed': sum(p.total_pages_printed for p in self.printers)
        }
        return stats
    
    def display_current_status(self):
        """Display current system status"""
        print("\n" + "="*60)
        print("PRINT QUEUE MANAGEMENT SYSTEM")
        print("="*60)
        
        print(f"\nQueue Status:")
        print(f"  High priority jobs: {self.priority_queue.size()}")
        print(f"  Normal jobs: {self.job_queue.size()}")
        print(f"  Completed jobs: {len(self.completed_jobs)}")
        
        print(f"\nPrinter Status:")
        for printer in self.printers:
            print(f"  {printer.get_status()}")
        
        # Show next few jobs in queue
        print(f"\nNext jobs in queue:")
        count = 0
        temp_queue = QueueUsingDeque()
        
        # Show priority jobs first
        while not self.priority_queue.is_empty() and count < 3:
            job = self.priority_queue.dequeue()
            print(f"  [HIGH] {job}")
            temp_queue.enqueue(job)
            count += 1
        
        # Restore priority queue
        while not temp_queue.is_empty():
            self.priority_queue.enqueue(temp_queue.dequeue())
        
        # Show normal jobs
        while not self.job_queue.is_empty() and count < 5:
            job = self.job_queue.dequeue()
            print(f"  {job}")
            temp_queue.enqueue(job)
            count += 1
        
        # Restore normal queue
        while not temp_queue.is_empty():
            self.job_queue.enqueue(temp_queue.dequeue())
        
        if count == 0:
            print("  No jobs in queue")
        
        # Show recent completed jobs
        print(f"\nRecent completed jobs:")
        for job in self.completed_jobs[-3:]:
            wait_time = job.get_wait_time()
            print_time = job.get_print_time()
            print(f"  {job.document_name} (waited {wait_time:.1f}s, printed {print_time:.1f}s)")
        
        print("="*60)


def printer_system_demo():
    # Create print queue manager with 2 printers
    manager = PrintQueueManager(num_printers=2)
    
    print("PRINTER JOB MANAGEMENT SYSTEM DEMO")
    print("-" * 40)
    
    # Submit some jobs
    print("\nSubmitting print jobs...")
    manager.submit_job("Quarterly_Report.pdf", 25, priority=1)
    manager.submit_job("Meeting_Minutes.docx", 5, priority=3)
    manager.submit_job("Research_Paper.pdf", 40, priority=2)
    manager.submit_job("Invoice.xlsx", 3, priority=4)
    manager.submit_job("Presentation.pptx", 15, priority=1)
    manager.submit_job("Photos.zip", 30, priority=5)
    
    # Display initial status
    manager.display_current_status()
    
    # Start printing (in background)
    print("\nStarting printers...")
    manager.start_printing()
    
    # Let it run for a bit
    print("\nPrinting in progress (waiting 10 seconds)...")
    time.sleep(10)
    
    # Submit more jobs while printing
    print("\nSubmitting more jobs while printing...")
    manager.submit_job("Resume.pdf", 2, priority=1)
    manager.submit_job("Manual.pdf", 50, priority=4)
    
    # Wait a bit more
    time.sleep(15)
    
    # Display final status
    manager.display_current_status()
    
    # Get statistics
    stats = manager.get_queue_stats()
    print("\nSystem Statistics:")
    for key, value in stats.items():
        key_formatted = key.replace('_', ' ').title()
        if isinstance(value, float):
            print(f"  {key_formatted}: {value:.2f}")
        else:
            print(f"  {key_formatted}: {value}")
    
    # Stop the system
    manager.stop_printing()


if __name__ == "__main__":
    printer_system_demo()

PRINTER JOB MANAGEMENT SYSTEM DEMO
----------------------------------------

Submitting print jobs...


NameError: name 'pagesS' is not defined

##  Project 2: Restaurant Order Management System

In [1]:
import time
from enum import Enum

class OrderStatus(Enum):
    PENDING = "Pending"
    PREPARING = "Preparing"
    READY = "Ready"
    SERVED = "Served"
    CANCELLED = "Cancelled"


class MenuItem:
    def __init__(self, item_id, name, category, preparation_time, price):
        self.item_id = item_id
        self.name = name
        self.category = category
        self.preparation_time = preparation_time  # in minutes
        self.price = price
    
    def __str__(self):
        return f"{self.name} (${self.price:.2f}) - {self.preparation_time}min"


class Order:
    def __init__(self, order_id, table_number):
        self.order_id = order_id
        self.table_number = table_number
        self.items = []
        self.status = OrderStatus.PENDING
        self.order_time = time.time()
        self.start_time = None
        self.completion_time = None
        self.total_price = 0
    
    def add_item(self, menu_item, quantity=1):
        for _ in range(quantity):
            self.items.append(menu_item)
            self.total_price += menu_item.price
    
    def get_total_preparation_time(self):
        if not self.items:
            return 0
        # Assuming items are prepared sequentially
        return sum(item.preparation_time for item in self.items)
    
    def __str__(self):
        item_count = len(self.items)
        return f"Order #{self.order_id} (Table {self.table_number}): {item_count} items, ${self.total_price:.2f}"


class Kitchen:
    def __init__(self, name):
        self.name = name
        self.current_order = None
        self.is_busy = False
        self.completed_orders = []
    
    def prepare_order(self, order):
        self.current_order = order
        self.is_busy = True
        order.status = OrderStatus.PREPARING
        order.start_time = time.time()
        
        prep_time = order.get_total_preparation_time()
        print(f"Kitchen '{self.name}': Preparing Order #{order.order_id} "
              f"(Table {order.table_number}), est. {prep_time} minutes")
        
        # Simulate preparation time (1 second = 1 minute for demo)
        time.sleep(prep_time)
        
        order.completion_time = time.time()
        order.status = OrderStatus.READY
        self.completed_orders.append(order)
        self.current_order = None
        self.is_busy = False
        
        print(f"Kitchen '{self.name}': Order #{order.order_id} is READY!")
        return order


class RestaurantOrderSystem:
    def __init__(self):
        self.order_queue = QueueUsingLinkedList()
        self.pending_orders = []
        self.served_orders = []
        self.menu = self._initialize_menu()
        self.order_counter = 1
        self.kitchens = [Kitchen("Main Kitchen"), Kitchen("Grill Station")]
    
    def _initialize_menu(self):
        menu_items = [
            MenuItem(1, "Margherita Pizza", "Main", 15, 12.99),
            MenuItem(2, "Spaghetti Carbonara", "Main", 20, 14.99),
            MenuItem(3, "Caesar Salad", "Appetizer", 10, 8.99),
            MenuItem(4, "Grilled Salmon", "Main", 25, 22.99),
            MenuItem(5, "Cheeseburger", "Main", 15, 10.99),
            MenuItem(6, "French Fries", "Side", 8, 4.99),
            MenuItem(7, "Chocolate Cake", "Dessert", 10, 6.99),
            MenuItem(8, "Ice Cream", "Dessert", 5, 4.99),
            MenuItem(9, "Soft Drink", "Beverage", 2, 2.99),
            MenuItem(10, "Coffee", "Beverage", 5, 3.99)
        ]
        return {item.item_id: item for item in menu_items}
    
    def place_order(self, table_number, items_with_quantity):
        """Place a new order"""
        order = Order(self.order_counter, table_number)
        self.order_counter += 1
        
        for item_id, quantity in items_with_quantity:
            if item_id in self.menu:
                order.add_item(self.menu[item_id], quantity)
            else:
                print(f"Warning: Item ID {item_id} not found in menu")
        
        self.order_queue.enqueue(order)
        self.pending_orders.append(order)
        
        print(f"Order placed: {order}")
        self._display_queue_status()
        return order.order_id
    
    def process_next_order(self):
        """Process the next order in queue"""
        if self.order_queue.is_empty():
            print("No orders to process")
            return None
        
        # Find available kitchen
        available_kitchen = None
        for kitchen in self.kitchens:
            if not kitchen.is_busy:
                available_kitchen = kitchen
                break
        
        if not available_kitchen:
            print("All kitchens are busy. Please wait...")
            return None
        
        # Get next order
        order = self.order_queue.dequeue()
        
        # Start preparation in available kitchen
        available_kitchen.prepare_order(order)
        return order
    
    def serve_order(self, order_id):
        """Mark an order as served"""
        for order in self.pending_orders:
            if order.order_id == order_id and order.status == OrderStatus.READY:
                order.status = OrderStatus.SERVED
                self.pending_orders.remove(order)
                self.served_orders.append(order)
                print(f"Order #{order_id} served to Table {order.table_number}")
                return True
        
        print(f"Order #{order_id} not found or not ready")
        return False
    
    def cancel_order(self, order_id):
        """Cancel a pending order"""
        # Check if order is in queue
        temp_queue = QueueUsingLinkedList()
        found = False
        
        while not self.order_queue.is_empty():
            order = self.order_queue.dequeue()
            if order.order_id == order_id:
                order.status = OrderStatus.CANCELLED
                print(f"Cancelled order: {order}")
                found = True
                break
            temp_queue.enqueue(order)
        
        # Restore queue
        while not temp_queue.is_empty():
            self.order_queue.enqueue(temp_queue.dequeue())
        
        # Also check pending orders
        if not found:
            for order in self.pending_orders:
                if order.order_id == order_id and order.status != OrderStatus.PREPARING:
                    order.status = OrderStatus.CANCELLED
                    self.pending_orders.remove(order)
                    print(f"Cancelled order: {order}")
                    found = True
                    break
        
        if not found:
            print(f"Order #{order_id} not found or cannot be cancelled")
        
        return found
    
    def get_order_status(self, order_id):
        """Get status of a specific order"""
        # Check in queue
        temp_queue = QueueUsingLinkedList()
        found_order = None
        
        while not self.order_queue.is_empty():
            order = self.order_queue.dequeue()
            if order.order_id == order_id:
                found_order = order
            temp_queue.enqueue(order)
        
        # Restore queue
        while not temp_queue.is_empty():
            self.order_queue.enqueue(temp_queue.dequeue())
        
        # Check in pending orders
        if not found_order:
            for order in self.pending_orders:
                if order.order_id == order_id:
                    found_order = order
                    break
        
        # Check in served orders
        if not found_order:
            for order in self.served_orders:
                if order.order_id == order_id:
                    found_order = order
                    break
        
        if found_order:
            wait_time = (time.time() - found_order.order_time) / 60  # in minutes
            return {
                'order_id': found_order.order_id,
                'table': found_order.table_number,
                'status': found_order.status.value,
                'wait_time': f"{wait_time:.1f} minutes",
                'total_price': f"${found_order.total_price:.2f}"
            }
        
        return None
    
    def _display_queue_status(self):
        """Display current queue status"""
        print(f"Orders in queue: {self.order_queue.size()}")
    
    def display_menu(self):
        """Display restaurant menu"""
        print("\n" + "="*60)
        print("RESTAURANT MENU")
        print("="*60)
        
        categories = {}
        for item in self.menu.values():
            if item.category not in categories:
                categories[item.category] = []
            categories[item.category].append(item)
        
        for category, items in categories.items():
            print(f"\n{category}:")
            for item in items:
                print(f"  {item.item_id:2d}. {item}")
    
    def display_system_status(self):
        """Display current system status"""
        print("\n" + "="*60)
        print("RESTAURANT ORDER MANAGEMENT SYSTEM")
        print("="*60)
        
        print(f"\nOrders in queue: {self.order_queue.size()}")
        print(f"Orders being prepared: {len([o for o in self.pending_orders if o.status == OrderStatus.PREPARING])}")
        print(f"Orders ready to serve: {len([o for o in self.pending_orders if o.status == OrderStatus.READY])}")
        print(f"Orders served today: {len(self.served_orders)}")
        
        print(f"\nKitchen Status:")
        for kitchen in self.kitchens:
            if kitchen.is_busy:
                print(f"  {kitchen.name}: Busy with Order #{kitchen.current_order.order_id}")
            else:
                print(f"  {kitchen.name}: Available")
        
        # Show next orders in queue
        print(f"\nNext orders in queue:")
        if self.order_queue.is_empty():
            print("  No orders in queue")
        else:
            # Display without dequeuing
            temp_queue = QueueUsingLinkedList()
            count = 0
            
            while not self.order_queue.is_empty() and count < 5:
                order = self.order_queue.dequeue()
                print(f"  #{order.order_id}: Table {order.table_number}, "
                      f"{len(order.items)} items, ${order.total_price:.2f}")
                temp_queue.enqueue(order)
                count += 1
            
            # Restore queue
            while not temp_queue.is_empty():
                self.order_queue.enqueue(temp_queue.dequeue())
        
        print("="*60)


def restaurant_system_demo():
    system = RestaurantOrderSystem()
    
    print("RESTAURANT ORDER MANAGEMENT SYSTEM DEMO")
    print("-" * 40)
    
    # Display menu
    system.display_menu()
    
    # Place some orders
    print("\nPlacing orders...")
    system.place_order(1, [(1, 1), (3, 1), (9, 2)])  # Table 1
    system.place_order(2, [(5, 2), (6, 2), (9, 2)])  # Table 2
    system.place_order(3, [(4, 1), (10, 1)])         # Table 3
    system.place_order(4, [(2, 1), (7, 2)])          # Table 4
    
    # Display system status
    system.display_system_status()
    
    # Process orders
    print("\nProcessing orders...")
    for _ in range(2):  # Process 2 orders
        system.process_next_order()
    
    # Check order status
    print("\nChecking order status...")
    status = system.get_order_status(1)
    if status:
        print(f"Order #{status['order_id']} status: {status['status']}")
    
    # Serve an order
    print("\nServing order...")
    system.serve_order(1)
    
    # Cancel an order
    print("\nCancelling order...")
    system.cancel_order(4)
    
    # Display final status
    system.display_system_status()
    
    # Place another order
    print("\nPlacing new order...")
    system.place_order(5, [(8, 4)])
    
    # Process remaining orders
    print("\nProcessing remaining orders...")
    while not system.order_queue.is_empty():
        system.process_next_order()
    
    # Final status
    system.display_system_status()


if __name__ == "__main__":
    restaurant_system_demo()

NameError: name 'QueueUsingLinkedList' is not defined

# 5. Common Queue Algorithms
## 1. Generate Binary Numbers

In [10]:
def generate_binary_numbers(n):
    """Generate first n binary numbers using queue"""
    if n <= 0:
        return []
    
    result = []
    queue = QueueUsingDeque()
    queue.enqueue("1")
    
    for _ in range(n):
        binary = queue.dequeue()
        result.append(binary)
        
        queue.enqueue(binary + "0")
        queue.enqueue(binary + "1")
    
    return result

In [12]:
# 2. Implement Stack using Two Queues

class StackUsingTwoQueues:
    def __init__(self):
        self.queue1 = QueueUsingDeque()
        self.queue2 = QueueUsingDeque()
    
    def push(self, x):
        """Push element onto stack"""
        self.queue2.enqueue(x)
        
        while not self.queue1.is_empty():
            self.queue2.enqueue(self.queue1.dequeue())
        
        # Swap queues
        self.queue1, self.queue2 = self.queue2, self.queue1
    
    def pop(self):
        """Remove element from top of stack"""
        if self.queue1.is_empty():
            return None
        return self.queue1.dequeue()
    
    def top(self):
        """Get top element"""
        if self.queue1.is_empty():
            return None
        return self.queue1.front()
    
    def is_empty(self):
        return self.queue1.is_empty()

In [14]:
# 3. First Non-Repeating Character in Stream
def first_non_repeating(stream):
    """Find first non-repeating character in stream"""
    queue = QueueUsingDeque()
    char_count = {}
    result = []
    
    for char in stream:
        # Update count
        char_count[char] = char_count.get(char, 0) + 1
        
        # Add to queue
        queue.enqueue(char)
        
        # Remove repeating characters from front
        while not queue.is_empty() and char_count[queue.front()] > 1:
            queue.dequeue()
        
        # Get first non-repeating
        if queue.is_empty():
            result.append(-1)
        else:
            result.append(queue.front())
    
    return result

# 6. Advanced Queue Types
## 1. Priority Queue (using heapq)

In [15]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.heap = []
        self.counter = 0  # To handle priority ties
    
    def enqueue(self, item, priority):
        """Add item with given priority"""
        heapq.heappush(self.heap, (priority, self.counter, item))
        self.counter += 1
    
    def dequeue(self):
        """Remove and return highest priority item"""
        if self.is_empty():
            raise IndexError("Dequeue from empty priority queue")
        priority, counter, item = heapq.heappop(self.heap)
        return item
    
    def is_empty(self):
        return len(self.heap) == 0
    
    def size(self):
        return len(self.heap)

In [16]:
# 2. Circular Queue
class CircularQueue:
    def __init__(self, capacity):
        self.capacity = capacity
        self.queue = [None] * capacity
        self.front = 0
        self.rear = -1
        self.size = 0
    
    def enqueue(self, item):
        """Add item to queue"""
        if self.is_full():
            raise IndexError("Queue is full")
        
        self.rear = (self.rear + 1) % self.capacity
        self.queue[self.rear] = item
        self.size += 1
    
    def dequeue(self):
        """Remove item from queue"""
        if self.is_empty():
            raise IndexError("Queue is empty")
        
        item = self.queue[self.front]
        self.front = (self.front + 1) % self.capacity
        self.size -= 1
        return item
    
    def is_empty(self):
        return self.size == 0
    
    def is_full(self):
        return self.size == self.capacity
    
    def peek(self):
        if self.is_empty():
            return None
        return self.queue[self.front]